In [1]:
import mobile_traffic as mt

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
paris = mt.get_city_traffic_data(traffic_type=mt.TrafficType.USERS, city=mt.City.PARIS, geo_data_type=mt.GeoDataType.IRIS) 

100%|██████████| 77/77 [01:20<00:00,  1.05s/it]


In [3]:
traffic_time_series = paris.get_traffic_time_series_by_location()

In [5]:
traffic_time_series.head()

iris,751010101,751010102,751010103,751010104,751010105,751010199,751010201,751010202,751010203,751010204,...,954270102,954270106,954270107,955270000,955550102,955550104,955550106,955820102,955820103,955850119
datetime,,,,,,,,,,,,,,,,,,,,,
2019-03-17 15:00:00,704634.966667,529145.163889,396799.046667,1.493695e+06,1.651012e+06,2.664653e+06,1.521427e+06,708795.488889,1.324204e+06,1.757763e+06,...,10423.281111,10751.062222,10821.023333,55467.028889,13797.100000,9865.967222,31503.108889,57389.787222,5143.897222,17636.890000
2019-03-17 15:15:00,699426.946667,563624.370556,412380.514444,1.553631e+06,1.554195e+06,2.659790e+06,1.481128e+06,695261.494444,1.172094e+06,1.638265e+06,...,9309.548333,10796.390556,11790.232222,58952.576667,12074.550556,7355.281111,24899.825000,57062.860000,5382.804444,18209.930000
2019-03-17 15:30:00,746904.470556,573484.660556,514600.927222,1.998304e+06,1.828856e+06,2.952835e+06,1.361535e+06,851561.327778,1.455008e+06,1.573832e+06,...,11138.257222,11329.157778,12679.111667,49352.250556,12433.309444,8195.643889,27210.586667,57265.909444,5401.019444,19416.045556
2019-03-17 15:45:00,853726.522222,631847.927222,445208.890000,1.752858e+06,1.991674e+06,3.050082e+06,1.420639e+06,803039.593333,1.312178e+06,1.632086e+06,...,11390.785000,10986.518889,10942.046111,47699.493889,12049.310000,7889.141111,26196.793333,55766.732222,5006.611667,17286.166667
2019-03-17 16:00:00,706118.302222,556488.347222,420536.647222,1.788563e+06,1.749459e+06,2.797334e+06,1.469390e+06,733603.863889,1.268962e+06,1.637837e+06,...,12274.115000,11047.979444,10110.787222,42104.445000,11124.768333,6420.969444,22113.921667,57871.610000,4936.757222,15574.785000


In [12]:
import plotly.graph_objs as go
import numpy as np

In [27]:
times = np.unique(traffic_time_series.index.time)
time_to_int = {time: index for index, time in enumerate(times)}
int_to_time = {index: time for index, time in enumerate(times)}

In [16]:
traffic_time_series['int_time'] = [time_to_int[i] for i in traffic_time_series.index.time]

In [17]:
traffic_time_series_mean = traffic_time_series.groupby(['int_time']).mean()

In [54]:
focal_times = list(range(86,96)) + list(range(0,16))

In [55]:
traffic_time_series_mean = traffic_time_series_mean.loc[focal_times]

In [56]:
traffic_time_series_mean = traffic_time_series_mean / traffic_time_series_mean.sum()

In [57]:
cum_sum = traffic_time_series_mean.cumsum(axis=0)

In [58]:
deviation = cum_sum.subtract(cum_sum.mean(axis=1), axis=0)

In [59]:
deviation.sum(axis=0).sort_values(ascending=False)

iris
914790104    2.046749
772820000    1.615766
772910000    1.523552
751020502    1.080612
751020501    1.075955
               ...   
930710107   -1.229936
930050702   -1.386957
940210101   -1.406055
930050703   -1.430050
930050701   -1.438450
Length: 2931, dtype: float64

In [61]:
fig = go.Figure()
x = [int_to_time[i] for i in traffic_time_series_mean.index]
y_low = traffic_time_series_mean['930050701']
y_high= traffic_time_series_mean['914790104']
y_mean = traffic_time_series_mean.mean(axis=1)
line_width = 4
fig.add_trace(go.Scatter(x=x, y=y_high, mode='lines', name='L1', line_width=line_width))
fig.add_trace(go.Scatter(x=x, y=y_low, mode='lines', name='L2', line_width=line_width))
fig.add_trace(go.Scatter(x=x, y=y_mean, mode='lines', name='Mean', line_width=line_width, line_color='black'))
fig.update_layout(title='', xaxis_title='Time', yaxis_title='Probability', template='plotly_white', font=dict(size=30, color="Black", family="Arial"), width=1500, height=800)
fig.show(renderer='browser')

In [62]:
import plotly.io as pio

In [63]:
path = '/Users/anmusso/Desktop/PhD/Projects/Current/NetMob/Figures/Sleep/'
pio.write_image(fig, file=f'{path}/poster_traffic_time_series.png', format='png', width=1500, height=800, scale=1)

In [65]:
L = 56
20.8 - 1.05 * L + 0.01486 * L**2

8.60096